#### multi-class classification: there are multiple possibilities of a target, like MNIST
#### multi-label classification: one target can have multiple labels, like many objects in one image

#### evaluation metrics:
1. precision at k (P@k)
2. average precision at k (AP@k)
3. mean average precision at k (MAP@k)
4. log loss

In [13]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import seaborn as sns 

from sklearn import datasets
from sklearn import manifold

%matplotlib inline

In [14]:
def pk(y_true, y_pred, k):
    """precision @ k"""
    if k==0:
        return 0
    
    y_pred = y_pred[:k]
    pred_set = set(y_pred)
    true_set = set(y_true)
    common_values = pred_set.intersection(true_set)
    return len(common_values)/k

In [15]:
def apk(y_true, y_pred, k):
    """average precision @ k"""
    pk_values = []
    for i in range(1, k+1):
        pk_values.append(pk(y_true[:i], y_pred[:i], i))
    
    if len(pk_values)==0:
        return 0
    return sum(pk_values)/len(pk_values)

In [16]:
y_true = [
    [1, 2, 3],
    [0, 2],
    [1],
    [2, 3],
    [1, 0],
    []
]

y_pred = [
    [0, 1, 2],
    [1],
    [0, 2, 3],
    [2, 3, 4, 0],
    [0, 1, 2],
    [0]
]

for i in range(len(y_true)):
    for j in range(1, 4):
        print(
            f"""
            y_true = {y_true[i]},
            y_pred = {y_pred[i]},
            AP@{j} = {apk(y_true[i], y_pred[i], k=j)}
            """
        )


            y_true = [1, 2, 3],
            y_pred = [0, 1, 2],
            AP@1 = 0.0
            

            y_true = [1, 2, 3],
            y_pred = [0, 1, 2],
            AP@2 = 0.25
            

            y_true = [1, 2, 3],
            y_pred = [0, 1, 2],
            AP@3 = 0.38888888888888884
            

            y_true = [0, 2],
            y_pred = [1],
            AP@1 = 0.0
            

            y_true = [0, 2],
            y_pred = [1],
            AP@2 = 0.0
            

            y_true = [0, 2],
            y_pred = [1],
            AP@3 = 0.0
            

            y_true = [1],
            y_pred = [0, 2, 3],
            AP@1 = 0.0
            

            y_true = [1],
            y_pred = [0, 2, 3],
            AP@2 = 0.0
            

            y_true = [1],
            y_pred = [0, 2, 3],
            AP@3 = 0.0
            

            y_true = [2, 3],
            y_pred = [2, 3, 4, 0],
            AP@1 = 1.0
            

            y_tru

In [17]:
def mapk(y_true, y_pred, k):
    """mean average precision @ k"""
    apk_values = []
    for i in range(len(y_true)):
        apk_values.append(apk(y_true[i], y_pred[i], k))
    return sum(apk_values)/len(apk_values)

In [18]:
y_true = [
    [1,2,3],
    [0,2],
    [1],
    [2,3],
    [1,0],
    []
]

y_pred = [
    [0,1,2],
    [1],
    [0,2,3],
    [2,3,4,0],
    [0,1,2],
    [0]
]

In [19]:
mapk(y_true=y_true, y_pred=y_pred, k=1)

0.16666666666666666

In [20]:
mapk(y_true=y_true, y_pred=y_pred, k=2)

0.2916666666666667

In [21]:
mapk(y_true=y_true, y_pred=y_pred, k=3)

0.3055555555555555

In [22]:
mapk(y_true=y_true, y_pred=y_pred, k=4)

0.2916666666666667

In [23]:
def apk(actual, predicted, k=10):
    if len(predicted) > k:
        predicted = predicted[:k]
    
    score = 0.0
    num_hits = 0.0
    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1
            score += num_hits/(i+1)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

In [24]:
apk(y_true, y_pred, 3)

0.16666666666666666

## regression

#### error = true_value - predicted_value
#### absolute error = |error|
#### squared error = (true value - predicted value)^2

In [27]:
def mean_absolute_error(y_true, y_pred):
    error = 0 
    for yt, yp in zip(y_true, y_pred):
        error += np.abs(yt-yp)
    return error / len(y_true)

In [28]:
def mean_squared_error(y_true, y_pred):
    error = 0
    for yt, yp in zip(y_true, y_pred):
        error += (yt-yp)**2
    return error/len(y_true)

#### RMSE (root mean squared error) = SQRT(mean squared error)

#### sqaured logarithmic error --> mean squared logarithmic error
#### root mean square logarithmic error (RMSLE)

In [29]:
def mean_squared_log_error(y_true, y_pred):
    error = 0 
    for yt, yp in zip(y_true, y_pred):
        error += (np.log(1+yt) - np.log(1+yp))**2
    return error / len(y_true)

#### precision error = (true value - predicted value) / true value * 100

In [30]:
def mean_percentage_error(y_true, y_pred):
    error = 0
    for yt, yp in zip(y_true, y_pred):
        error += (yt-yp)/yt
    return error / len(y_true)

#### $R^2$: coefficient of determination

$$
R^2 = 1 - \frac{\sum (y_{ti} - y_{pi})^2}{\sum (y_{ti} - y_{tmean})^2}
$$

In [31]:
def r2(y_true, y_pred):
    mean_true_value = np.mean(y_true)
    numerator = 0
    denominator = 0 
    for yt, yp in zip(y_true, y_pred):
        numerator += (yt - yp)**2
        denominator += (yt-mean_true_value)**2
    ratio = numerator / denominator
    return 1 - ratio

In [32]:
def mae_np(y_true, y_pred):
    return np.mean(np.abs(y_true-y_pred))

In [33]:
# QWK: quadratic weighted kappa, Cohen's kappa

from sklearn import metrics

y_true = [1, 2, 3, 1, 2, 3, 1, 2, 3]
y_pred = [2, 1, 3, 1, 2, 3, 3, 1, 2]

print(metrics.cohen_kappa_score(y_true, y_pred, weights="quadratic"))
print(metrics.accuracy_score(y_true, y_pred))

0.33333333333333337
0.4444444444444444


#### Matthew's Correlation Coefficient (MCC)

$$
MCC = \frac{TP*TN - FP*FN}{\sqrt{(TP+FP)*(FN+TN)*(FP+TN)*(TP+FN)}}
$$

In [34]:
def true_positive(y_true, y_pred):
    tp = 0
    for yt, yp in zip(y_true, y_pred):
        if yt==1 and yp==1:
            tp += 1
    return tp

def true_negative(y_true, y_pred):
    tn = 0
    for yt, yp in zip(y_true, y_pred):
        if yt==0 and yp==0:
            tn += 1
    return tn 

def false_positive(y_true, y_pred):
    fp = 0
    for yt, yp in zip(y_true, y_pred):
        if yt==0 and yp==1:
            fp += 1
    return fp

def false_negative(y_true, y_pred):
    fn = 0
    for yt, yp in zip(y_true, y_pred):
        if yt==1 and yp==0:
            fn += 1
    return fn 

In [ ]:
def mcc(y_true, y_pred):
    tp = true_positive(y_true, y_pred)
    tn = true_negative(y_true, y_pred)
    fp = false_positive(y_true, y_pred)
    fn = false_negative(y_true, y_pred)

    numerator = (tp * tn) / (fp * fn)
    demoninator = (
        (tp + fp) *
        (fn + tn) * 
        (fp + tn) * 
        (tp + fn)
    )
    return numerator / demoninator**0.5